# Example of preprocessing

In [ ]:
import hep_eeg.hep as hep

### default parameters

In [ ]:
# default parameters
subject_list = [1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,21,22,101]
rename_channel = {"FPz":"Fpz","OZ":"Oz"}
bads = {16:['C3']}

### Get trigger IDs from CSV file

In [ ]:
# Get trigger IDs from CSV file
triggers = hep.get_triggers_id("trigger.csv")
triggers

### set up your own pipeline

In [ ]:
# set up your own pipeline
def pipeline(subject_id,bads=None):
    xdf_path = f"data/raw/{subject_id:02d}.xdf"
    # load file and remove dc
    raw = hep.load_xdf_to_raw(xdf_path,
        rename_dict=rename_channel).remove_ecg_dc().remove_resp_dc()
    # interpolate bads channels
    if bads:
        raw = raw.interpolate_bads(bads)
    # detect ecg peaks
    ecg_events = raw.detect_peaks_to_events(event_id = 9)
    # add ecg pecks to events
    raw = raw.add_events(ecg_events)
    # bandpass
    raw = raw.filter(0.1,40)
    # extract epochs
    epochs = raw.extract_epochs((-0.1, 0.8),event_id = 9,label_list=[triggers])
    # apply baseline
    epochs = epochs.apply_baseline(bl_windows=[-0.1,0])
    # reject bad epochs
    epochs = epochs.reject_bad_epochs()
    # apply re-reference
    epochs = epochs.apply_reref()
    return epochs

### preprocessing and save all subjects

In [ ]:
# preprocessing and save all subjects
for subject_id in subject_list:
    print(f"Processing subject {subject_id}")
    # use the pipeline
    epochs = pipeline(subject_id,bads = bads[subject_id])
    # save epochs
    epochs.save(f"data/epochs/epochs-{subject_id}.pickle")

### run ICA

In [ ]:
# preprocessing and save all subjects
for subject_id in subject_list:
    print(f"Processing subject {subject_id}")
    # use the pipeline
    epochs = hep.load(f"data/epochs/epochs-{subject_id}.pickle")
    # run ICA and exclude bad components
    ica = epochs.run_ica(plot=False)
    bads_ica = epochs.auto_find_bad_ica(ica)
    print(f"Subject {subject_id} bad ICA components: {bads_ica}")
    epochs = epochs.exclude_ica(ica, bads_ica)
    # save epochs
    epochs.save(f"data/epochs/epochs-{subject_id}-ica.pickle")